In [1]:
import gensim
from sklearn.model_selection import train_test_split
from collections import namedtuple
from gensim.models import doc2vec, word2vec
from gensim.models.fasttext import FastText

In [2]:
from analysis import *
from tqdm import tqdm
file_path = './data/docs_jsonl_space_jamo.txt'

In [3]:
app_id_list, app_name_list, cate_list, rating_list, docs = read_jsonl(file_path, key_ma=False, 
                                                                      only_ma=False)

Reading Files: 100%|██████████| 616311/616311 [00:03<00:00, 157485.01it/s]


In [4]:
from konlpy.tag import Mecab
mecab = Mecab()

In [5]:
def stopwords(docs, tokenizer):
    new_docs = []
    for doc in tqdm(docs, desc='doing', total=len(docs)):
        remove_list = list(set([ma[0] for ma in tokenizer.pos(doc) if ma[1] in ['SF', 'SC', 'SSO', 'SSC', 'SC', 'SY']]))
        for r in remove_list:
            doc = doc.replace(r, '').replace('   ', ' ').replace('  ', ' ')
        new_docs.append(doc)
        
    return new_docs

In [6]:
new_docs = stopwords(docs, mecab)

doing: 100%|██████████| 616311/616311 [01:09<00:00, 8857.46it/s] 


In [7]:
file_path2 = './data/docs_jsonl_remove_S.txt'

In [22]:
save_jsonl(file_path2, app_id_list, app_name_list, cate_list, rating_list, new_docs, key_ma=False)

Saving documents: 100%|██████████| 616311/616311 [00:06<00:00, 95731.89it/s] 


In [38]:
file_path = './data/docs_jsonl_ma_mecab.txt'
app_id_list, app_name_list, cate_list, rating_list, docs = read_jsonl(file_path, key_ma=True, 
                                                                      only_ma=False)

Reading Files: 100%|██████████| 616311/616311 [01:06<00:00, 9313.80it/s] 


In [39]:
docs_morphs = []
for doc in docs:
    docs_morphs.append([ma[0] for ma in doc])

TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_docs_morphs = [TaggedDocument(d, [r]) for d, r in zip(docs_morphs, app_id_list)]
tagged_docs_morphs = [TaggedDocument(d, [r]) for d, r in zip(docs, app_id_list)]

In [42]:
with open('./data/docs_mecab_noma.txt', 'w', encoding='utf-8') as file:
    for doc in docs_morphs:
        print(' '.join(doc), file=file)

FastText

등록되지 않은 데이터를 처리하기 위함

In [44]:
ft_model = FastText.load_fasttext_format('./data/gensim_model/fasttext/mecab_model')

In [50]:
test_word = '복음'
print('{0} exists'.format(test_word)) if test_word in ft_model.wv.index2word else print('{0} not exists'.format(test_word))
ft_model.most_similar(negative=[test_word])

복음 exists


[('내려요', 0.4601520299911499),
 ('여서', 0.36375096440315247),
 ('으니까', 0.3515215218067169),
 ('튕김현상', 0.3389400243759155),
 ('당합니다', 0.3371686339378357),
 ('들으면', 0.3327321708202362),
 ('질립니다', 0.32110482454299927),
 ('올라서', 0.319903165102005),
 ('라이트', 0.31211328506469727),
 ('터집니다', 0.3097568154335022)]

In [ ]:
file_path = './data/docs_jsonl_ma_mecab.txt'
app_id_list, app_name_list, cate_list, rating_list, docs = read_jsonl(file_path, key_ma=True, 
                                                                      only_ma=False)
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_docs_morphs = [TaggedDocument(d, [r]) for d, r in zip(docs, app_id_list)]

In [77]:
dictionary = gensim.corpora.Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]
tf_idf = gensim.models.TfidfModel(corpus)